In [ ]:
%pip install spotipy google-api-python-client google-auth-oauthlib google-auth-httplib2 --upgrade

In [ ]:
%pip install pandas

In [ ]:
# import required dependencies
import os
from IPython.display import display, Markdown

import spotipy
from spotipy.oauth2 import SpotifyOAuth

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

import pandas as pd

In [ ]:
SPOTIFY_CLIENT_ID = ""      # Spotify client id goes here
SPOTIFY_CLIENT_SECRET = ""  # Spotify client secret goes here

In [ ]:

# generate the oauth credentials for the user
def establish_google_credentials():
    scopes = ["https://www.googleapis.com/auth/youtube"]
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    client_secrets_file = "creds.json"
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    
    credentials = flow.run_local_server()
    return credentials

In [ ]:
# create a youtube_api object
def youtube_api(credentials):
    api_service_name = "youtube"
    api_version = "v3"
    youtube = googleapiclient.discovery.build(api_service_name, api_version, credentials=credentials)
    return youtube

In [ ]:
# save google credentials
credentials = establish_google_credentials()

In [ ]:
# create the youtube api object and save to variable
yt_api = youtube_api(credentials)

In [ ]:
# get the first 25 playlists a user has
def get_youtube_playlists():
    request = yt_api.playlists().list(
        part="snippet,contentDetails",
        maxResults=25, # change this value here if your desired playlist is not in the first batch of 25. Increase to 50 to get the next 25 results. So on and so forth until you find what your looking for
        mine=True
    )

    response = request.execute()
    return response

In [ ]:
# get the youtube playlists
youtube_playlists = get_youtube_playlists()

In [ ]:
# creates a pandas dataframe to display the youtube playlists
def list_yt_playlists():
    columns = [
        "Channel",
        "Title",
        "Description",
        "ID"
    ]

    data = []

    items = youtube_playlists.get("items")
    for playlist in items:
        id = playlist.get("id")
        snippet = playlist.get("snippet")
        localized = snippet.get("localized")
        title = localized.get("title")
        description = localized.get("description")
        channel_title = snippet.get("channelTitle")

        temp_arr = [channel_title, title, description, id]
        data.append(temp_arr) 

    df = pd.DataFrame(columns=columns, data=data)
    return df

In [ ]:
# actually list the playlists
user_youtube_playlists = list_yt_playlists()
display(user_youtube_playlists)

In [ ]:
# prompt the user to enter the playlist they want to use
def choose_yt_playlist():
    index = int(input("Enter the line number of the playlist. This is the number following the channel column... "))
    display(user_youtube_playlists.iloc[[index]])
    return user_youtube_playlists.loc[index, "ID"]

choosen_youtube_playlist = choose_yt_playlist()

In [ ]:
# create the spotify credentials
def establish_spotify_credentials():
    scope = "user-library-read"
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET, redirect_uri="http://localhost:3000"))

    return sp

In [ ]:
# save spotify credentials
spotify_credentials = establish_spotify_credentials()

In [ ]:
# function to get the spotify playlists
def get_spotify_playlists(offset=0):
    return spotify_credentials.current_user_playlists(limit=25, offset=offset)

In [ ]:
# creates a pandas dataframe to list the spotify playlists
def list_spotify_playlists():
    offset = 0 # if you don't see your playlist, increase this number by 25 until you do
    playlists = get_spotify_playlists(offset).get("items")

    columns = [
        "Owner",
        "Title",
        "Description",
        "ID"
    ]

    data = []

    for playlist in playlists:
        id = playlist.get("id")
        name = playlist.get("name")
        description = playlist.get("description")
        owner = playlist.get("owner")
        display_name = owner.get("display_name")

        temp_arr = [display_name, name, description, id]
        data.append(temp_arr)

    df = pd.DataFrame(columns=columns, data=data)
    return df

In [ ]:
# do the listing
spotify_playlists = list_spotify_playlists()
display(spotify_playlists)

In [ ]:
# prompt the user to choose the playlist to use
def choose_spotify_playlist():
    index = int(input("Enter the line number of the playlist. This is the number following the owner column... "))
    display(spotify_playlists.iloc[[index]])
    return spotify_playlists.loc[index, "ID"]

choosen_spotify_playlist = choose_spotify_playlist()

In [ ]:
# retrive songs in the spotify playlist. gets 100 by default
def get_songs_from_playlist(limit=100, offset=0):
    data = spotify_credentials.playlist_items(playlist_id=choosen_spotify_playlist, limit=limit, offset=offset)
    songs = data.get("items")
    total = data.get("total")
    return [songs, total]

In [ ]:
# loops through to get every song until they have been stored
def generate_spotify_songs_list():
    has_next = True
    offset = 0
    limit = 100

    data = []

    while True:
        songs, total = get_songs_from_playlist(limit=limit, offset=offset)
        has_next = len(data) + len(songs) < int(total)
        data = [*data, *songs]

        print(f'Fetched {len(data)} out of {int(total)} songs')

        offset = offset + limit

        if not has_next:
            break

    return data

In [ ]:
# save all the songs in the spotify playlist to a variable
spotify_playlist_songs = generate_spotify_songs_list()

In [ ]:
# function to search youtube with a keyword phrase. gets the first search result. returns whole resposne
def search_youtube(keywords):
    request = yt_api.search().list(
        part = "snippet",
        maxResults=1,
        q=keywords
    )

    response = request.execute()
    
    return response

In [ ]:
# method to add the song to the youtube playlist
def add_song_to_playlist(videoId, position):
    request = yt_api.playlistItems().insert(
        part="snippet",
        body={
          "snippet": {
            "playlistId": choosen_youtube_playlist,
            "position": position,
            "resourceId": {
              "kind": "youtube#video",
              "videoId": videoId
            }
          }
        }
    )

    response = request.execute()

In [ ]:
# actually adds the songs to a youtube playlist
def add_songs_to_youtube_playlist():
    failed_songs = []
    searched_songs_count = 0
    # loop through each song in the spotify playlist
    for song in spotify_playlist_songs:
        # get track
        track = song.get("track")

        # get tracks title and artists
        title = track.get("name")
        artists = track.get("artists")
        
        # generate a keyword search string
        artists_name_str = ", ".join([artist.get("name") for artist in artists])
        keyword_search_string = f"{title} by {artists_name_str} official audio"
        
        # try to search
        try: 
            # search youtube for the video using the keyword string as the query
            youtube_search_response = search_youtube(keyword_search_string)

            # if nothing is found, move on to the next song
            if int(youtube_search_response.get("pageInfo").get("totalResults")) == 0:
                failed_songs.append(keyword_search_string)
                continue
            
            # take the first item. the search function returns 1 item but a list is still returned
            youtube_items = youtube_search_response.get("items")
            video_id = youtube_items[0].get("id").get("videoId")

            # call function to add song to youtube playlist
            add_song_to_playlist(video_id, searched_songs_count)
            searched_songs_count = searched_songs_count + 1
        # continue to next song on error
        except Exception as e:
            failed_songs.append(keyword_search_string)
            continue

    print(f"Successfully uploaded {searched_songs_count} out of {len(spotify_playlist_songs)} songs")

    if(len(failed_songs) > 0):
        print("Failed to upload the following songs")
        for song in failed_songs:
            print(song)

In [ ]:
# adds the songs to a youtube playlist
add_songs_to_youtube_playlist()